In [1]:
# 필요한 기본 패키지 준비

# 데이터 처리 필요 패키지
import numpy as np
import pandas as pd
import datetime as dt

# 시각화 필요 패키지
%matplotlib inline
from plotnine import *
import folium
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rc, font_manager
import seaborn as sns


# Machine Learning 분석 환경 준비

# 전처리, 스케일링
from sklearn.preprocessing import StandardScaler

# 선형회귀분석
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from scipy import stats

# OLS회귀분석
import statsmodels.api as sm

# GAM 일반화가법모형
# LinearGAM, LogisticGAM, PoissonGAM, GammaGAM, InvGuss
from pygam import LinearGAM, LogisticGAM, PoissonGAM, GammaGAM

# Boosting

#데이터셋 분리
from sklearn.model_selection import train_test_split



# 한글 처리
font_name = font_manager.FontProperties(fname='C:/Windows/Fonts/NanumGothicCoding.ttf').get_name()
rc('font',family=font_name)

# - 마이너스 사인 처리
matplotlib.rcParams['axes.unicode_minus'] = False

# jupyter notebook에서 warning 무시하기
import warnings
warnings.filterwarnings("ignore")

In [16]:
#########################################################################
# MAD 기반 예제코드
def mad_based_outlier(points, thresh=3.5):
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points, axis=0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)
    modified_z_score = 0.6745 * diff / med_abs_deviation
    return modified_z_score > thresh 

# 출처: https://pythonanalysis.tistory.com/7 [Python 데이터 분석]
#########################################################################

# 소셜 데이터 처리를 위한 함수
# 1. 모든 소셜 데이터 column들의 첫번째는 : 날짜다.
# 2. 각 소셜데이터는 social_키워드.블로그/트위터/뉴스/총합 으로 되어 있다.
def changeColNames(d,to_replace, replaced) : 
    # 컬럼이름 리스트를 만들어 반환
    # 통합하기 쉽게, 모든 데이터들의 날짜컬럼 이름을 date로 통일
    new_col_names = ['date']
    new_col_names.extend(list(d.columns)[1:])
    d.columns = new_col_names
    return pd.Series(d.columns).apply(lambda x : x.replace(to_replace,replaced))


# ['temp','humid','wind','rain','snow','cloud','sun_time'
#                  ,'pm.total', 'health.total','br.total', 'hobby.total','date.total']
# modeling 함수로 만들어 처리하기
def linReg(df, item, cols_using):
    cols = cols_using
    X = df.loc[df['category']==item, cols]
    y = df.loc[df['category']==item,'qty']

    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=0)

    model = LinearRegression().fit(X_train, y_train)
  
    print('LinearRegression을 이용한 %s의 회귀분석 결과 :'%item)
    print('훈련세트점수 : {:.2f}'.format(model.score(X_train, y_train)))
    print('검증세트점수 : {:.2f}'.format(model.score(X_test, y_test)))

    
def ridgeReg(df, item, cols_using):
    cols = cols_using
    X = df.loc[df['category']==item,cols]
    y = df.loc[df['category']==item,'qty']

    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=0)

    ridge = Ridge(alpha=0.1, normalize=True, random_state=0, tol=0.001).fit(X_train, y_train)
    
    print('RidgeRegression을 이용한 %s의 회귀분석 결과 :'%item)
    print('훈련세트점수 : {:.2f}'.format(ridge.score(X_train, y_train)))
    print('검증세트점수 : {:.2f}'.format(ridge.score(X_test, y_test)))


def lassoReg(df, item, cols_using):
    cols = cols_using
    X = df.loc[df['category']==item,cols]
    y = df.loc[df['category']==item,'qty']

    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=0)

    lasso = Lasso(alpha=0.1, max_iter=1000).fit(X=X_train, y=y_train)
  
    print('LassoRegression을 이용한 %s의 회귀분석 결과 :'%item)
    print('훈련세트점수 : {:.2f}'.format(lasso.score(X_train, y_train)) )
    print('검증세트점수 : {:.2f}'.format(lasso.score(X_test, y_test)) )

    #사용한 특성수
    print('사용한 특성수 : {}'.format(np.sum(lasso.coef_ != 0)) )
    
def addDayWeek(df):
    df_work = df.copy()
    df_work['day'] = pd.Series(range(1,df_work.shape[0]+1))
    df_work['week'] = df_work['day'].apply(lambda x : x//7)
    return df_work
    
def mergeForAnalysis(df1, df2, df3, item):
    merged_df = pd.merge(df1.loc[df1.category==item], df2, on='date',how='left')
    merged_df = pd.merge(merged_df, df3, on='date', how='left')
    return merged_df

In [17]:
# 데이터 불러오기 (전처리 된 GS, 랄라블라, 날씨)
gs = pd.read_csv('d:/project/contest/data/processed/p_gs.csv', parse_dates=['date'])
lv = pd.read_csv('d:/project/contest/data/processed/p_lavla.csv', parse_dates=['date'])
w = pd.read_csv('d:/project/contest/data/processed/p_wUVair_seoul.csv', parse_dates=['date'], index_col=0)
sns_all = pd.read_csv('d:/project/contest/data/processed/social_all.csv', parse_dates=['date'])

In [18]:
# GS/lv 서울시만
gs_seoul = gs.loc[gs.pvn_nm =='서울특별시']
lv_seoul = lv.loc[lv.pvn_nm =='서울특별시']

In [19]:
# 컬럼 이름을 Series로 돌려주는 함수를 자체 개발함
sns_all.columns = changeColNames(sns_all,'.','_')
# 컬럼명의 마침표(.)가 전부 '_'로 변경되었다.
# ols formula입력시 오류 발생 방지
# sns_all.columns

In [20]:
cols_to_keep = ['date','bor_nm','gender','age_cd','category','qty']
gs_grouped = gs_seoul[cols_to_keep].groupby(by=['date','bor_nm','category']).sum().reset_index()
gs_grouped.tail(2)

,date,bor_nm,category,qty
289910,2018-12-31,중랑구,아이스크림,161
289911,2018-12-31,중랑구,탄산음료,63


In [31]:
lv_grouped = lv_seoul[cols_to_keep].groupby(by=['date','bor_nm','category']).sum().reset_index()
lv_grouped.tail(2)

,date,bor_nm,category,qty
188498,2018-12-31,중랑구,크림로션,182
188499,2018-12-31,중랑구,훼이셜클렌저,292


In [33]:
# 서울특별시 단위로 df조정(구단위 데이터를 시단위로 합치기)
day_gs_grouped = gs_grouped.groupby(by=['date','category']).sum().reset_index()
day_gs_grouped.tail(3)

,date,category,qty
12991,2018-12-31,스타킹,918
12992,2018-12-31,아이스크림,6149
12993,2018-12-31,탄산음료,3089


In [34]:
# 서울특별시 단위로 df조정(구단위 데이터를 시단위로 합치기)
day_lv_grouped = lv_grouped.groupby(by=['date','category']).sum().reset_index()
day_lv_grouped.tail(3)

,date,category,qty
10955,2018-12-31,체중조절,760
10956,2018-12-31,크림로션,7359
10957,2018-12-31,훼이셜클렌저,5314


In [35]:
# '아이스크림'만 골라서 날씨('w')df와 소셜('sns_all')df와 결합해 새로운 'day_gs_grouped_w_item'생성
item = '아이스크림'
day_gs_grouped_w_item = mergeForAnalysis(day_gs_grouped, w, sns_all, item)
# day_gs_grouped_w_item = pd.merge(day_gs_grouped_w.loc[day_gs_grouped_w.category==item],w,on='date',how='left')
# day_gs_grouped_w_item = pd.merge(day_gs_grouped_w_item,sns_all,on='date',how='left')
day_gs_grouped_w_item.tail(3)

,date,category,qty,loc,temp,rain,cloud,wind,humid,hpa,...,date_news,date_total,br_blog,br_twitter,br_news,br_total,hobby_blog,hobby_twitter,hobby_news,hobby_total
1093,2018-12-29,아이스크림,7264,108,-7.6,0.0,0.0,2.9,29.0,1024.3,...,80,2617,4325,996,54,5375,700,1280,8,1988
1094,2018-12-30,아이스크림,6758,108,-6.8,0.0,0.8,1.2,35.9,1026.8,...,35,2825,4679,1172,44,5895,696,1314,16,2026
1095,2018-12-31,아이스크림,6149,108,-5.4,0.0,1.6,1.2,39.6,1026.2,...,46,3034,5167,1349,54,6570,965,1441,38,2444


In [23]:
# ind_vars = ['temp','humid','wind','sun_time']
# ind_vars = ['temp','humid','wind','rain','snow','cloud','sun_time']
# ind_vars = ['temp','humid','wind','rain','snow','cloud','sun_time'
#                 ,'pm_total', 'health_total','br_total', 'hobby_total','date_total']

ind_vars = ['temp','humid','wind','rain','snow','cloud','sun_time'
             ,'pm_total', 'health_total','br_total', 'hobby_total','date_total']

linReg(day_gs_grouped_w_item,item,ind_vars)
ridgeReg(day_gs_grouped_w_item,item,ind_vars)
lassoReg(day_gs_grouped_w_item,item,ind_vars)

LinearRegression을 이용한 아이스크림의 회귀분석 결과 :
훈련세트점수 : 0.81
검증세트점수 : 0.79
RidgeRegression을 이용한 아이스크림의 회귀분석 결과 :
훈련세트점수 : 0.80
검증세트점수 : 0.79
LassoRegression을 이용한 아이스크림의 회귀분석 결과 :
훈련세트점수 : 0.81
검증세트점수 : 0.79
사용한 특성수 : 12


In [24]:
Xy = day_gs_grouped_w_item.loc[day_gs_grouped_w_item['category']==item,ind_vars+['qty']]
# model = sm.OLS.from_formula("qty ~ temp + sun_time + rain + snow + cloud", data=Xy)
model = sm.OLS.from_formula("qty ~ temp + humid + sun_time + rain", data=Xy)
# model = sm.OLS.from_formula("qty ~ temp", data=Xy)

print(model.fit().summary())

                            OLS Regression Results                            
Dep. Variable:                    qty   R-squared:                       0.762
Model:                            OLS   Adj. R-squared:                  0.761
Method:                 Least Squares   F-statistic:                     874.9
Date:                Thu, 11 Jul 2019   Prob (F-statistic):               0.00
Time:                        11:24:09   Log-Likelihood:                -11089.
No. Observations:                1096   AIC:                         2.219e+04
Df Residuals:                    1091   BIC:                         2.221e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1.963e+04   2250.516     -8.725      0.0

In [25]:
gs_items = gs.category.unique()
lv_items = lv.category.unique()

In [36]:
# 한번에 처리 gs
for item in gs_items:
    print('\n',item,' 판매량 분석 - 모든 독립변수 포함(날씨+sns(합계))')
    df_working = mergeForAnalysis(day_gs_grouped, w, sns_all, item)
    linReg(df_working,item,ind_vars)
    ridgeReg(df_working,item,ind_vars)
    lassoReg(df_working,item,ind_vars)


 라면  판매량 분석 - 모든 독립변수 포함(날씨+sns(합계))
LinearRegression을 이용한 라면의 회귀분석 결과 :
훈련세트점수 : 0.23
검증세트점수 : 0.18
RidgeRegression을 이용한 라면의 회귀분석 결과 :
훈련세트점수 : 0.23
검증세트점수 : 0.18
LassoRegression을 이용한 라면의 회귀분석 결과 :
훈련세트점수 : 0.23
검증세트점수 : 0.18
사용한 특성수 : 12

 과자  판매량 분석 - 모든 독립변수 포함(날씨+sns(합계))
LinearRegression을 이용한 과자의 회귀분석 결과 :
훈련세트점수 : 0.45
검증세트점수 : 0.41
RidgeRegression을 이용한 과자의 회귀분석 결과 :
훈련세트점수 : 0.44
검증세트점수 : 0.41
LassoRegression을 이용한 과자의 회귀분석 결과 :
훈련세트점수 : 0.45
검증세트점수 : 0.41
사용한 특성수 : 12

 마스크  판매량 분석 - 모든 독립변수 포함(날씨+sns(합계))
LinearRegression을 이용한 마스크의 회귀분석 결과 :
훈련세트점수 : 0.79
검증세트점수 : 0.80
RidgeRegression을 이용한 마스크의 회귀분석 결과 :
훈련세트점수 : 0.78
검증세트점수 : 0.76
LassoRegression을 이용한 마스크의 회귀분석 결과 :
훈련세트점수 : 0.79
검증세트점수 : 0.80
사용한 특성수 : 12

 맥주  판매량 분석 - 모든 독립변수 포함(날씨+sns(합계))
LinearRegression을 이용한 맥주의 회귀분석 결과 :
훈련세트점수 : 0.61
검증세트점수 : 0.55
RidgeRegression을 이용한 맥주의 회귀분석 결과 :
훈련세트점수 : 0.60
검증세트점수 : 0.55
LassoRegression을 이용한 맥주의 회귀분석 결과 :
훈련세트점수 : 0.61
검증세트점수 : 0.55
사용한 특성수 : 12

 생리대  판매량 분석 - 모든 독립변수 포함(날씨+sn

In [37]:
# 한번에 처리 lv
for item in lv_items:
    print('\n',item,' 판매량 분석 - 모든 독립변수 포함')
    df_working = mergeForAnalysis(day_lv_grouped, w, sns_all, item)
    linReg(df_working,item,ind_vars)
    ridgeReg(df_working,item,ind_vars)
    lassoReg(df_working,item,ind_vars)


 립컬러  판매량 분석 - 모든 독립변수 포함
LinearRegression을 이용한 립컬러의 회귀분석 결과 :
훈련세트점수 : 0.08
검증세트점수 : 0.01
RidgeRegression을 이용한 립컬러의 회귀분석 결과 :
훈련세트점수 : 0.07
검증세트점수 : 0.02
LassoRegression을 이용한 립컬러의 회귀분석 결과 :
훈련세트점수 : 0.08
검증세트점수 : 0.01
사용한 특성수 : 12

 립케어  판매량 분석 - 모든 독립변수 포함
LinearRegression을 이용한 립케어의 회귀분석 결과 :
훈련세트점수 : 0.39
검증세트점수 : 0.39
RidgeRegression을 이용한 립케어의 회귀분석 결과 :
훈련세트점수 : 0.39
검증세트점수 : 0.38
LassoRegression을 이용한 립케어의 회귀분석 결과 :
훈련세트점수 : 0.39
검증세트점수 : 0.39
사용한 특성수 : 12

 마스크팩  판매량 분석 - 모든 독립변수 포함
LinearRegression을 이용한 마스크팩의 회귀분석 결과 :
훈련세트점수 : 0.03
검증세트점수 : -0.00
RidgeRegression을 이용한 마스크팩의 회귀분석 결과 :
훈련세트점수 : 0.03
검증세트점수 : 0.00
LassoRegression을 이용한 마스크팩의 회귀분석 결과 :
훈련세트점수 : 0.03
검증세트점수 : -0.00
사용한 특성수 : 12

 바디로션  판매량 분석 - 모든 독립변수 포함
LinearRegression을 이용한 바디로션의 회귀분석 결과 :
훈련세트점수 : 0.34
검증세트점수 : -0.01
RidgeRegression을 이용한 바디로션의 회귀분석 결과 :
훈련세트점수 : 0.33
검증세트점수 : -0.00
LassoRegression을 이용한 바디로션의 회귀분석 결과 :
훈련세트점수 : 0.34
검증세트점수 : -0.01
사용한 특성수 : 12

 체중조절  판매량 분석 - 모든 독립변수 포함
LinearRegression을 이용한 체중조절의